In [1]:
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchsummary import summary
import torch
import torch.nn.functional as F
import numpy as np
import importlib 
import config
importlib.reload(config)

<module 'config' from 'e:\\TSAI\\EMLO\\EMLO-session1\\config.py'>

In [2]:
class DatasetFromArray(Dataset):
    def __init__(self, featurepath, labelpath):
        self.data = np.load(open(featurepath, 'rb'))
        self.labels = np.load(open(labelpath, 'rb'))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return {'data': torch.tensor(self.data[index], dtype=torch.float32), 'target': torch.tensor(self.labels[index],dtype=torch.long)}
    

In [3]:
train_ds = DatasetFromArray(config.TRAIN_FEATURES, config.TRAIN_LABELS)
valid_ds = DatasetFromArray(config.VALID_FEATURES, config.VALID_LABELS)

In [4]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=16, pin_memory=True)

In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(512, 64, kernel_size=3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.5),
            
            nn.Conv2d(64,2, kernel_size=3, bias=False),
            nn.AvgPool2d(3)

        )
    
    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 2)
        return F.log_softmax(x, dim=1)

In [6]:

model = Model().to('cuda')
summary(model, input_size=(512,7,7))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 5, 5]         294,912
              ReLU-2             [-1, 64, 5, 5]               0
       BatchNorm2d-3             [-1, 64, 5, 5]             128
           Dropout-4             [-1, 64, 5, 5]               0
            Conv2d-5              [-1, 2, 3, 3]           1,152
         AvgPool2d-6              [-1, 2, 1, 1]               0
Total params: 296,192
Trainable params: 296,192
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.10
Forward/backward pass size (MB): 0.05
Params size (MB): 1.13
Estimated Total Size (MB): 1.27
----------------------------------------------------------------


In [7]:
del model

In [8]:
from tqdm import tqdm
from functools import partial
import torch.nn.functional as F
tqdm = partial(tqdm, leave=True, position=0)
model = Model()
model.to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.CrossEntropyLoss().to('cuda')





def train(model, train_loader, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    
    for _, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data['data'].to('cuda'), data['target'].to('cuda')

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += loss.detach()
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)
        # print(f'Output : {output[0:5]}')
        # print(f'Print: {pred[0:5]}')
        # print(f'Target: {target[0:5]}')
        correct += pred.eq(target.view_as(pred)).sum().item()
    
    # return train_loss, correct
    print(
            f"Training loss = {train_loss * 1.0 / len(train_loader.dataset)},Training Accuracy : {100.0 * correct / len(train_loader.dataset)}"
        )

def evaluate(model, valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    with torch.no_grad():
        for _, data in tqdm(enumerate(valid_loader), total=len(valid_loader)):
            data, target = data['data'].to('cuda'), data['target'].to('cuda')
            output = model(data)
            valid_loss +=F.nll_loss(output, target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    print(
            f"Valid loss = {valid_loss * 1.0 / len(valid_loader.dataset)}, Valid Accuracy : {100.0 * correct / len(valid_loader.dataset)}"
        )

for i in range(10):
    print(f'EPOCH {i+1}')
    train(model, train_loader, optimizer)
    evaluate(model, valid_loader)


EPOCH 1


100%|██████████| 63/63 [00:02<00:00, 28.06it/s]


Training loss = 0.0191042497754097,Training Accuracy : 86.4


100%|██████████| 50/50 [00:00<00:00, 147.93it/s]


Valid loss = 0.261130005903542, Valid Accuracy : 88.25
EPOCH 2


...

Valid loss = 0.3656344138015993, Valid Accuracy : 89.25
EPOCH 10


100%|██████████| 63/63 [00:00<00:00, 212.84it/s]


Training loss = 0.0005314643494784832,Training Accuracy : 99.7


100%|██████████| 50/50 [00:00<00:00, 284.62it/s]

Valid loss = 0.37610316054939175, Valid Accuracy : 90.25
